# Sesión 4.1. Teoría - Redes Neuronales Recurrentes Avanzadas
Curso 2022-23

Profesor: [Jorge Calvo Zaragoza](mailto:jcalvo@dlsi.ua.es)

## Resumen
En esta sesión:
  * Profundizaremos en el uso de redes recurrentes en Keras.
  * Introduciremos el encaje vectorial de palabras (*word embeddings*)
  * Explicaremos mecanismos adicionales que facilitan el uso de RNN en Keras (funciones generadoras).

## Entradas discretas

El *one-hot encoding* es una representación numérica de variables discretas. Consiste en asumir vectores de características de dimension igual al número de diferentes valores del conjunto discreto de la variable a codificar. Cada dimension corresponde a un único elemento de dicho conjunto. Hasta ahora las hemos usado para la salida de la red, pero **se puede usar también para la entrada**.


Normalmente las redes recurrentes se usan para tareas de *procesamiento de lenguaje natural*. En este tipo de problemas, los elementos de entrada son  elementos discretos como caracteres o palabras, y no características numéricas. Si una red neuronal sólo *entiende* de números, ¿cómo podemos indicarle este tipo de características en la entrada? Del mismo modo que se indica para las categorías: utilizando una codificación one-hot.

El one-hot encoding tiene una particularidad importante: no asume ninguna relación entre los elementos del vocabulario. Dicho de otro modo, todos los elementos del conjunto son equidistantes entre sí en la codificación one-hot (distancia hamming 2). Por otra parte, el one-hot encoding tiene la máxima redundancia; en realidad, para codificar $N$ elementos, tan sólo harían falta vectores de $log(N)$ bits.





In [1]:
"""

Asumamos el principio de un ejercicio en el cual
la red va a predecir sobre secuencias que indican
colores.

"""


import numpy as np

# Definimos el vocabulario de colores
vocabulary = {'rojo', 'amarillo', 'azul', 'verde', 'lila' ,'naranja'}

# Asignamos a cada palabra un índice numérico
word_to_int = dict([(char, i) for i, char in enumerate(vocabulary)])
print('Asignacion de indice a palabras')
print(word_to_int)

# De esta forma las frases se componen de secuencias de vectores de 6 elementos
sentence = 'rojo amarillo naranja'
tokenized_sentence = sentence.split()
encoded_sentence = np.zeros([len(tokenized_sentence),len(vocabulary)])

# Para cada palabra, activamos la posición que corresponde a su identificador
for i,c in enumerate(sentence.split()):
  encoded_sentence[i][ word_to_int[c] ] = 1

# Comprobación
print()
print('Frase original: {}'.format(sentence))

print()
print('Frase secuencial:')
print(str(tokenized_sentence))

print()
print('Frase codificada:')
print(str(encoded_sentence))

Asignacion de indice a palabras
{'azul': 0, 'naranja': 1, 'verde': 2, 'amarillo': 3, 'lila': 4, 'rojo': 5}

Frase original: rojo amarillo naranja

Frase secuencial:
['rojo', 'amarillo', 'naranja']

Frase codificada:
[[0. 0. 0. 0. 0. 1.]
 [0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0.]]



#### Estado actual

A menudo encontramos tareas para las cuales el vocabulario a tener en cuenta es inmenso. Por ejemplo, el idioma español tiene alrededor de 100.000 vocablos distintos. Además, ¿qué ocurre con las palabras muy poco frecuentes o que incluso están fuera del vocabulario que habíamos planeado inicialmente?

Una posibilidad para lidiar con este escenario es bajar a nivel de caracteres, cuyo vocabulario suele ser mucho más restringido. El problema es que la red neuronal tiene que hacer un esfuerzo mayor en inferir las relaciones entre los distintos elementos de la entrada. En la actualidad, es común utilizar un enfoque intermedio basado en sub-palabras, con criterios tomados de teoría de la información.

## Encaje de palabras (Word Embedding)

El one-hot encoding permite al usuario proporcionar a la red una entrada en la cual cada elemento tiene una representación equidistante con respecto a cualquier otro. Obviamente, para una tarea específica, esta asunción no es valida (ni útil). Por ejemplo, en problmas de *procesamiento de lenguaje natural* hay palabras que tienen un rol o un significado similar y por tanto, sería adecuado que tuvieran una representación interna similar.

Idealmente, queremos representar cada palabra mediante un vector numérico en el cual las palabras similares (en rol o en significado) esten cerca en ese espacio. Esto se conoce como *embedding*.  Siguiendo los principios del deep learning, queremos que la red aprenda estas relaciones por sí sola en lugar de establecer el embedding siguiendo reglas heurísticas o basadas en nuestra propia intuición.






### word2vec

La idea de realizar realizar embeddings es muy antigua. Sin embargo, su popularidad se incrementó considerablemente a partir del surgimiento de los modelos *word2vec*.


#### Idea


![texto alternativo](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/img/Bottou-WordSetup.png)

#### Análisis

![texto alternativo](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/img/Colbert-WordTable2.png)

#### Representación semántica

[Visualizacion](http://metaoptimize.s3.amazonaws.com/cw-embeddings-ACL2010/embeddings-mostcommon.EMBEDDING_SIZE=50.png)

![texto alternativo](http://colah.github.io/posts/2014-07-NLP-RNNs-Representations/img/Mikolov-GenderVecs.png)

### Capas de embedding en Keras

En Keras, la capa de embedding no se hace a través de operaciones matemáticas. En su lugar, lo que se hace es acceder a una tabla look-up que asocia cada entero con una posición. La capa Embedding de Keras recibe directamente el entero que identifica un elemento del vocabulario (sin necesidad de hacer un one-hot encoding) y devuelve su representación densa. Los parámetros de la capa son:
* **input_dim**: tamaño del vocabulario de entrada (por ejemplo, número de palabras del problema)
* **output_dim**: tamaño del espacio vectorial donde se encaja la entrada

In [2]:
%%capture --no-stdout

import tensorflow as tf
import numpy as np


# Definimos el tamaño del vocabulario y las dimensiones del espacio latente
tam_vocabulario = 5
tam_embedding   = 8

# Definimos una entrada de longitud variable
capa_entrada = tf.keras.layers.Input(shape=(None,), dtype='int32')

# Añadimos la capa de embedding
embedding = tf.keras.layers.Embedding(input_dim=tam_vocabulario,
                      output_dim=tam_embedding)(capa_entrada)

# Creamos un modelo a partir de estas dos capas
model = tf.keras.models.Model(capa_entrada,embedding)
model.summary()

# Probamos a 'predecir' a través de esta red
codificacion_entera = [4,1,3,3,3]
codificacion_embedding = model.predict(np.asarray([codificacion_entera]))

print()
print('Representación de {}'.format( str(codificacion_entera) ))
print(codificacion_embedding)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 8)           40        
                                                                 
Total params: 40
Trainable params: 40
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 167ms/step

Representación de [4, 1, 3, 3, 3]
[[[ 0.04419135  0.03557061  0.00836201 -0.03705425  0.01063796
    0.02537375  0.01366648  0.00534002]
  [-0.02562686 -0.01458812  0.02063059 -0.01800523 -0.04325141
   -0.03176136  0.01700329  0.02383772]
  [ 0.00808215  0.03995105  0.00221574  0.03086218  0.00888181
    0.00838411  0.01129188 -0.01288737]
  [ 0.00808215  0.03995105  0.00221574  0.03086218  0.00888181
 

A pesar de esta implementación, los valores de esta tabla se aprenden de manera específica para la tarea en cuestión con los medios convencionales de entrenamiento de redes neuronales.

A menudo es interesante importar los embeddings obtenidos mediante un word2vec de un dominio similar para utilizarlo como punto de partida en nuestra tarea.

Keras facilita este escenario permitiendo establecer los pesos iniciales de la capa mediante el parámetro *weights*. Además, nos permite especificar si queremos modificar los pesos en nuestro propio entrenamiento o no.



In [3]:
from keras.models import Model
from keras.layers import Input, Embedding
import numpy as np

# Definimos el tamaño del vocabulario y las dimensiones del espacio latente
tam_vocabulario = 5
tam_embedding   = 3

# Creamos una matriz de embedding aleatoria
matriz_embedding = np.random.rand(tam_vocabulario,tam_embedding)

print('Tabla de embedding inicial')
for idx, representation in enumerate(matriz_embedding):
  print(idx,representation)

# Definimos una entrada de longitud variable
capa_entrada = tf.keras.layers.Input(shape=(None,), dtype='int32')

# Proporcionamos la matriz anterior como parámetro
embedding = tf.keras.layers.Embedding(input_dim=tam_vocabulario,
                      output_dim=tam_embedding,
                      weights=[matriz_embedding],
                      trainable=True)(capa_entrada)

# Creamos el modelo de embedding
model = tf.keras.models.Model(capa_entrada,embedding)

# Comprobamos el embedding sobre este modelo
codificacion_entera = [4,1,3,3,3]
codificacion_embedding = model.predict(np.asarray([codificacion_entera]))

print()
print('Representación de {}'.format( str(codificacion_entera) ))
print(codificacion_embedding)

Tabla de embedding inicial
0 [0.58007216 0.26285934 0.64265632]
1 [0.71907505 0.20183993 0.14741692]
2 [0.35635058 0.47093649 0.16693704]
3 [0.73443321 0.98123054 0.09198138]
4 [0.16951729 0.21098193 0.34760449]
1/1 [==============================] - 0s 48ms/step

Representación de [4, 1, 3, 3, 3]
[[[0.1695173  0.21098194 0.34760448]
  [0.719075   0.20183994 0.14741692]
  [0.73443323 0.98123056 0.09198138]
  [0.73443323 0.98123056 0.09198138]
  [0.73443323 0.98123056 0.09198138]]]


### Funciones generadoras

En el caso de las tareas resueltas con RNN es habitual tener secuencias de longitud variable. Aunque, por definición, las RNN pueden manejarlas, internamente Keras trabaja con lotes de datos de dimensión fija para un procesamiento más eficiente.

Para aliviar este problema, hay dos alternativas: crear un lote por cada secuencia (lotes de 1 muestra), lo cual es ineficiente, o usar la técnica de *padding*. El *padding* (relleno) consiste en calcular la secuencia más larga y establecer las dimensiones de acuerdo con este valor. Las secuencias más cortas se rellenan con valores nulos (normalmente 0).

El problema es que, cuando las diferencias entre las longitudes son muy altas, el efecto del *padding* es muy severo y causa un aprendizaje menos efectivo. Sin embargo, hay una solución intermedia, elegante, para hacer frente a este problema: los mini-lotes (*mini batches*). Es decir, construir pequeños lotes y aplicar *padding* en cada mini-lote de forma independiente.

Para poder hacer esto en Keras, es necesario usar funciones *generadoras*. Las *generadoras* son funciones de Python que preparan tales lotes. En cada llamada, el generador prepara el siguiente lote para ser considerado y se congela hasta que se vuelve a llamar para generar el siguiente lote. Las funciones del generador nos permiten devolver lotes que contienen un número fijo de elementos. Luego podemos hacer el relleno a nivel de lote (*intra-batch padding*), reduciendo así el problema mencionado anteriormente.

El siguiente código proporciona un ejemplo intuitivo de este comportamiento (prestad especial atención a las palabras reservadas **yield** y **next**).

In [4]:
import numpy as np

# Conjunto de datos de muestras de longitud variable
muestra = [[1,2], [3,4,5], [6,7,8,9], [10]]

# Funcion generadora
def ejemplo_generador(data, tam_lote):

  while True:
      # Creamos lotes de tamaño `tam_lote`
      for idx in range(0,len(data),tam_lote):

        # Datos de este lote
        lote = data[idx:idx+tam_lote] # data[0:2) -> 0,1, data[2:4) -> 2,3

        # Calculamos longitud maxima
        longitud_maxima = max([len(b) for b in lote])

        # Creamos un lote de dimensiones fijas y relleno de 0
        lote_dimension_fija = np.zeros((len(lote),longitud_maxima),
                                       dtype=int)


        # Ponemos los datos reales en el lote secuencia a secuencia
        for idx_s, secuencia in enumerate(lote):
          # Y dato a dato
          for idx_d, dato in enumerate(secuencia):
            lote_dimension_fija[idx_s,idx_d] = dato

        yield lote_dimension_fija


# Inspeccionamos la muestra
print('Muestra')
print(muestra)


# Creamos el generador
generador = ejemplo_generador(muestra, tam_lote = 2)

# Inspeccionamos la salida del generador
print()
print('Lote',1)
x  = next(generador)
print(x)
print('Forma {}'.format( x.shape ))
print()
print('Lote',2)
x  = next(generador)
print(x)
print('Forma {}'.format( x.shape ))


Muestra
[[1, 2], [3, 4, 5], [6, 7, 8, 9], [10]]

Lote 1
[[1 2 0]
 [3 4 5]]
Forma (2, 3)

Lote 2
[[ 6  7  8  9]
 [10  0  0  0]]
Forma (2, 4)


#### Bucketting

Otro concepto relacionado es *bucketting*, que complementa al *padding* para hacerlo más adecuado al entrenar una red neuronal. Esta estrategia primero ordena el conjunto de datos para que los elementos dentro del mismo lote tengan una longitud similar. Esto minimiza el *padding* intra-lote, por lo que se acerca a la forma óptima (pero ineficiente) de realizar el proceso de entrenamiento secuencia a secuencia.

In [5]:
import numpy as np

# Conjunto de datos de muestras de longitud variable
muestra = [[1,2], [3,4,5], [6,7,8,9], [10]]

# Funcion generadora que implementa bucketting
def bucketting_generador(data, tam_lote):

    data.sort(key = len) # [!] Ordenamos los datos primero

    while True: # Infinita -> sirve datos continuamente
      # Aqui podriamos re-barajar los datos

      # Creamos lotes de tamaño `tam_lote`
      for idx in range(0,len(data),tam_lote):

        # Datos de este lote
        lote = data[idx:idx+tam_lote]

        # Calculamos longitud maxima
        longitud_maxima = max([len(b) for b in lote])

        # Creamos un lote de dimensiones fijas y relleno de 0
        lote_dimension_fija = np.zeros((len(lote),longitud_maxima),
                                       dtype=int)


        # Ponemos los datos reales en el lote secuencia a secuencia
        for idx_s, secuencia in enumerate(lote):
          # Y dato a dato
          for idx_d, dato in enumerate(secuencia):
            lote_dimension_fija[idx_s,idx_d] = dato

        yield lote_dimension_fija


# Inspeccionamos la muestra
print('Muestra')
print(muestra)

# Creamos el generador
generador = bucketting_generador(muestra, tam_lote = 2)

# Inspeccionamos la salida del generador
print()
print('Lote',1)
x  = next(generador)
print(x)
print('Forma {}'.format( x.shape ))
print()
print('Lote',2)
x  = next(generador)
print(x)
print('Forma {}'.format( x.shape ))


Muestra
[[1, 2], [3, 4, 5], [6, 7, 8, 9], [10]]

Lote 1
[[10  0]
 [ 1  2]]
Forma (2, 2)

Lote 2
[[3 4 5 0]
 [6 7 8 9]]
Forma (2, 4)


#### Keras y funciones generadoras

Keras permite integrar de forma natural las funciones generadoras en el proceso de entrenamiento mediante la función **fit** (en versiones anteriores  de Keras se llamaba a *fit_generator*). Esta función acepta una función generadora y va cargando los mini-lotes según se van generando. En este caso, los parámetros de la función **fit** son diferentes a los convencionales (como veremos más adelante).

## Ejemplo de RNN con funciones generadoras

En este ejemplo vamos a entrenar una red que detecte si una palabra ha sido generada aleatoriamente o proviene de un vocabulario. Para ello:

* Vamos a descargar un vocabulario de palabras de la web.
* Vamos a asignar un número a cada caracter.
* Vamos a implementar una función generadora que nos sirva lotes de palabras de la misma longitud.
  * Tanto palabras del vocabulario como generadas aleatoriamente.
* Vamos a definir el modelo de red RNN.
* Vamos a entrenar la red y evaluarlo interactivamente.

In [6]:
import numpy as np
import string
import re
import random
import urllib.request


# Funcion que nos devuelve palabras de un vocabualrio
def vocabulario_palabras():
  url_palabras = "https://svnweb.freebsd.org/csrg/share/dict/words?view=co&content-type=text/plain"
  palabras = urllib.request.urlopen(urllib.request.Request(url_palabras, headers={'User-Agent': 'Mozilla/5.0'})).read().decode().splitlines()

  # Filtramos las palabras que solo tengan caracteres alfabeticos y estén en minúsculas
  palabras_seleccionadas  = [palabra for palabra in palabras
                             if re.match('^[a-zA-Z]+$',palabra) and palabra.islower()]

  return palabras_seleccionadas


# Definimos el vocabulario a partir de la función anterior
vocabulario = vocabulario_palabras()
print()
print('No. palabras: {}'.format(len(vocabulario)))

# Nuestras categorías son las minusculas + símbolo de padding
conjunto_caracteres = 'P' + string.ascii_lowercase
tam_conjunto_caracteres = len(conjunto_caracteres)

print()
print('No. de caracteres: {} ({})'.format(tam_conjunto_caracteres,conjunto_caracteres))

# Creamos los conversores de char a int y viceversa
char_to_int = dict([(char, i) for i, char in enumerate(conjunto_caracteres)])
int_to_char = dict([(i, char) for i, char in enumerate(conjunto_caracteres)])

print()
print('Char to int: {}'.format(char_to_int))
print('Int to char: {}'.format(int_to_char))


No. palabras: 20409

No. de caracteres: 27 (Pabcdefghijklmnopqrstuvwxyz)

Char to int: {'P': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
Int to char: {0: 'P', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


Definimos la función generadora de lotes para entrenar la red:
* Asumimos que las variables anteriores son accesibles (ahorramos parámetros)
* Tendremos dos tipos de palabras: inventadas (0) y reales (1)
* Las palabras inventadas se generarán en esta misma función
* Las palabras reales se tomarán del vocabulario descargado y filtrado.

Después vamos a inspeccionar la función generadora para asegurarnos de que hace lo que se espera.

In [7]:

def generador(tam_lote = 32):
  # Bucle principal de la generadora
  while True:
    X  = []               # Variable donde guardar las palabras
    Y  = []               # Variable donde guardar las etiquetas

    # Cada lote tiene 'tam_lote' palabras
    for _ in range(tam_lote):

      if random.random() < 0.5:     # Al 50 % tomamos una real o una inventada

        # Generamos una cadena aleatoria de tamaño 3-8 (sin padding!)
        palabra = ''.join(random.choice(string.ascii_lowercase)
                          for i in range(random.randint(3,8)))
        etiqueta = False

      else:
        # Usar real
        palabra = random.choice(vocabulario)
        etiqueta = True

      X.append(palabra)
      Y.append(etiqueta)

    # Preparamos el lote
    max_longitud = max([len(palabra) for palabra in X])

    X_lote = np.zeros((tam_lote, max_longitud), dtype=np.int32)
    Y_lote = np.zeros((tam_lote, 1), dtype=np.int32)

    for idx_p, palabra in enumerate(X):
      for idx_c, caracter in enumerate(palabra):
          X_lote[idx_p][idx_c] = char_to_int[caracter] # 'a' -> 1, 'b' -> 2, etc.

    for idx_e, etiqueta in enumerate(Y):
      Y_lote[idx_e] = 1 if etiqueta == True else 0


    yield X_lote, Y_lote


# ----------------------------------------------------------------------
# Prueba de la función generadora

g = generador(2)
X_lote, Y_lote = next(g)
print(X_lote[0])
print(Y_lote[0])

print('Forma del lote X: {}'.format(X_lote.shape))
print('Forma del lote Y: {}'.format(Y_lote.shape))


[ 9 14 22  5 18 20  9  2 12  5]
[1]
Forma del lote X: (2, 10)
Forma del lote Y: (2, 1)


A continuación vamos a construir la RNN y a entrenarla. Para aprender este problema de secuencias vamos a crear una red con dos capas:

* La primera capa estará formada por 16 neuronas LSTM.
* La segunda capa será de tipo *Dense* y tendrá una única neurona (suficiente para problemas de clasificación binaria)
* La activación de la segunda capa será de tipo *Sigmoid*.

In [8]:
%%capture --no-stdout

# ------------------------------------
# Parámetros del problema

TAM_LOTES = 32
NUM_CARACTERISTICAS = tam_conjunto_caracteres

# ------------------------------------
# Modelo de red
print()
print('Construimos el modelo de red...')

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.InputLayer((None,))) # `None`significa que no es fijo !
model.add(tf.keras.layers.Embedding(input_dim=tam_conjunto_caracteres, output_dim=4))
model.add(tf.keras.layers.LSTM(32,return_sequences=True))
model.add(tf.keras.layers.LSTM(16))
model.add(tf.keras.layers.Dense(1))
model.add(tf.keras.layers.Activation('sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='rmsprop')
model.summary()

# ------------------------------------
# Entrenamiento
print()
print('Entrenamos la red...')

gen_training = generador(TAM_LOTES)
gen_validacion = generador(TAM_LOTES)

# La funcion `fit` tiene particularidades...
history = model.fit(gen_training,
                    steps_per_epoch=32,
                    validation_data=gen_validacion,
                    validation_steps=32,
                    epochs=30,
                    verbose=2)



Construimos el modelo de red...
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, None, 4)           108       
                                                                 
 lstm (LSTM)                 (None, None, 32)          4736      
                                                                 
 lstm_1 (LSTM)               (None, 16)                3136      
                                                                 
 dense (Dense)               (None, 1)                 17        
                                                                 
 activation (Activation)     (None, 1)                 0         
                                                                 
Total params: 7,997
Trainable params: 7,997
Non-trainable params: 0
_________________________________________________________________

Entrenamos la red...

Por último vamos a evaluar el modelo de red con los pesos aprendidos. Para esto usaremos nuevas secuencias y comprobaremos si la predicción es correcta.

In [9]:
%%capture --no-stdout

# -----------------------------------------------------------------
# Función que evalua una palabra
# - Asumimos que todas las variables anteriores son accesibles
# - Convertimos la palabra de entrada en un lote unitario
# - Predecimos utilizando la red

def evaluate(palabra):
  x = np.zeros((1, len(palabra)),
               dtype=np.int32)

  for idx_c, caracter in enumerate(palabra):
    x[0][idx_c] = char_to_int[caracter]


  prediccion = model.predict(x, verbose=0)[0]

  print('Nivel de `realismo` de {}: {}'.format(palabra,prediccion))


# Vamos a evaluar la siguiente secuencias
evaluate("d")
evaluate("housing")
evaluate("adfjkljk")


Nivel de `realismo` de d: [0.43565106]
Nivel de `realismo` de housing: [0.8638271]
Nivel de `realismo` de adfjkljk: [0.02914312]


&nbsp;

&nbsp;


---

 Vamos a practicar &#10158; [Ejercicio de predicción de opinion de una película](https://colab.research.google.com/drive/1r2P6ySoSRe36YJbO2kFtY7sxvOdvoRLj?usp=sharing)





---